In [39]:
import torch

print(torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
print(device)
#device=torch.device('cpu')########

1.3.1
cuda:0


In [40]:
import os

corpus_name = "train_model32000.txt"#"mecab_tagged.txt"#
position = "./data"
corpus = os.path.join(position, corpus_name)
print(corpus)

./data/train_model32000.txt


In [41]:
from Voc import *

In [42]:
printLines(corpus)

['▁なんと !', '▁なんと !\n'] 2 2
['▁なんと !', '▁ 意味 : 都 を 移 すこ と 、 都 移 り\n'] 2 12
['▁わたし 安定の たっくん ですよー 気づいたら たっくん いないし 、 いつき あるし 誰 と交換 したのか', '▁まさかの 増えてる w 羨ましい\n'] 13 4
['▁まさかの 増えてる w 羨ましい', '▁ 連れ が 誰か 引いて た んですよ 、 きっと それ と交換 したんだ と思うんですけど 誰 だったのか w あ 、 たっくん は しれっと 買取 に 出しました ー\n'] 4 25
['▁ おち つけ よ', '▁みんな は げろ\n'] 4 3
['▁みんな は げろ', '▁昨日 髪の毛 切った ばっかり だけど このくらい つる っ パ ゲ になって きます !\n'] 3 13
['▁今は 古 すぎて 廃 盤 のもの が多いので 、 いい モンスター が出る の だと 高い 値 で 売れ ます w', '▁あ 、 やっぱり そう言う のある ですね w\n'] 19 7
['▁あ 、 やっぱり そう言う のある ですね w', '▁ あります あります w\n'] 7 4
['▁やだ お', '▁ 目元 は 知ってる 、 かあいい\n'] 2 6
['▁ 目元 は 知ってる 、 かあいい', '▁超 過去 写真\n'] 6 3


In [43]:
MIN_COUNT = 28#22#3#
print(corpus_name)
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name)#, datafile, save_dir)
pairs = trimRareWords(voc, pairs, MIN_COUNT)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

train_model32000.txt
Start preparing training data ...
Reading lines...
Read 2187212 sentence pairs
Trimmed to 1834337 sentence pairs
Counting words...
Counted words: 34296
keep_words 31995 / 34293 = 0.9330
Trimmed from 1834337 pairs to 1822478, 0.9935 of total

pairs:
['▁なんと !', '▁なんと !']
['▁なんと !', '▁ 意味 : 都 を 移 すこ と 、 都 移 り']
['▁わたし 安定の たっくん ですよー 気づいたら たっくん いないし 、 いつき あるし 誰 と交換 したのか', '▁まさかの 増えてる w 羨ましい']
['▁ おち つけ よ', '▁みんな は げろ']
['▁みんな は げろ', '▁昨日 髪の毛 切った ばっかり だけど このくらい つる っ パ ゲ になって きます !']
['▁今は 古 すぎて 廃 盤 のもの が多いので 、 いい モンスター が出る の だと 高い 値 で 売れ ます w', '▁あ 、 やっぱり そう言う のある ですね w']
['▁あ 、 やっぱり そう言う のある ですね w', '▁ あります あります w']
['▁やだ お', '▁ 目元 は 知ってる 、 かあいい']
['▁ 目元 は 知ってる 、 かあいい', '▁超 過去 写真']
['▁ は ?', '▁クソ じゃん 気にし ねー で いいんじゃね w まあ 俺は ファイヤー 党 の人 なので もちろん あの方 に入れた が な']


In [44]:
import pickle
# #Load batches for each iteration
# train_pairs,test_pairs = train_test_split(pairs ,test_size=0.2)
# with open('data/train_test_pairs_mecab.pickle', 'wb') as f:
#     pickle.dump((train_pairs,test_pairs), f)
# with open("data/train_test_pairs_mecab3min.pickle", "rb") as f:
#     train_pairs,test_pairs = pickle.load(f)
with open("data/train_test_pairs_32k.pickle", "rb") as f:
    train_pairs,test_pairs = pickle.load(f)
 
print(voc.name)
print("ちゃんと目的のコーパスファイルを読み込んでいるか確認")
print("train size",len(train_pairs))
print("example",train_pairs[:2])
print("test size",len(test_pairs))
print("example",test_pairs[:2])

train_model32000.txt
ちゃんと目的のコーパスファイルを読み込んでいるか確認
train size 1457982
example [['▁ 問い合わせ w 間違いない ですね w', '▁ して はいない んですけどね w それも 運 なの だろう'], ['▁ 珈琲 とか ウイスキー 用', '▁ 娘 用']]
test size 364496
example [['▁ ピカチュウ は かわいい でも カビゴン はもっと かわいい', '▁いやあ ずみ のほうが'], ['▁ スカ トロ とか やめてくれ だめ や', '▁そこまで 言って ねーよ w それは あかん']]


In [45]:
# from model import *
from model_lstm import *

# model.pyのモデルの説明

Get embedding of current input word.
Forward through unidirectional GRU.
Calculate attention weights from the current GRU output from (2).
Multiply attention weights to encoder outputs to get new “weighted sum” context vector.
Concatenate weighted context vector and GRU output using Luong eq. 5.
Predict next word using Luong eq. 6 (without softmax).
Return output and final hidden state.

input_step: one time step (one word) of input sequence batch; shape=(1, batch_size)(1,3)
last_hidden: final hidden layer of GRU; shape=(n_layers x num_directions, batch_size, hidden_size)(1,3,2)
encoder_outputs: encoder model’s output; shape=(max_length, batch_size, hidden_size)(4,3,2)

In [25]:
#訓練するときだけ
print(corpus_name)
# Configure models
model_name = "focal_loss_model"
attn_model = 'dot'
# attn_model = 'general'
# attn_model = "concat"
hidden_size = 256
encoder_n_layers = 4
decoder_n_layers = 4
dropout = 0.1
batch_size = 512
teacher_forcing_ratio=0.2

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 300000
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                           '{}-{}_{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size, teacher_forcing_ratio),
                           '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    print("load save file")
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    # checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint["en"]
    decoder_sd = checkpoint["de"]
    encoder_optimizer_sd = checkpoint["en_opt"]
    decoder_optimizer_sd = checkpoint["de_opt"]
    embedding_sd = checkpoint["embedding"]
    voc.__dict__ = checkpoint["voc_dict"]


print("Building encoder and decoder ...")
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = AttnDecoderRNN(
    attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout
)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print("Models built and ready to go!")

train_model32000.txt
load save file
Building encoder and decoder ...
Models built and ready to go!


In [26]:
# パラメータカウント関数
print("パラメータ数をカウント")
print("encoderのパラメータ数")
parameters_count(encoder)
print("decoderパラメータ数")
parameters_count(decoder)
#print(encoder)

パラメータ数をカウント
encoderのパラメータ数
13975040
decoderパラメータ数
18651646


# 評価用pyファイルをインポート

In [27]:
# from eval_ja_seq2seq import *
# from eval_ja_seq2seq import mecabNormalizeString as normalizeString
# mecabOrSp = 'mecab'
from eval_lstm_ja_seq2seq import *
from eval_lstm_ja_seq2seq import sentencePieceNormalizeString as normalizeString
mecabOrSp = 'sentencePiece'

# 自分の入力で評価(会話)

In [29]:
encoder.eval()

decoder.eval()

# Initialize search module
#searcher = GreedySearchDecoder(encoder, decoder)
searcher = BeamSearchDecoder(encoder, decoder, 5, 5)
searcher１ = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, searcher1, voc, mecabOrSp)

>  q


# 複数文をランダムで入力して出力して評価

In [31]:
import random
num=5
test_choice = [(random.choice(test_pairs)[0].replace(' ',''),random.choice(test_pairs)[1].replace(' ','')) for i in range(num)]

encoder.eval()
decoder.eval()
# Initialize search module
#searcher = GreedySearchDecoder(encoder, decoder)
searcher = BeamSearchDecoder(encoder, decoder, 5, 5)

for input_pair in test_choice:
    
    input_sentence, output_sentence = input_pair
    print("入力:",input_sentence)
    input_sentence = normalizeString(input_sentence)
    print('入力を分割すると:',input_sentence)
    

    # Evaluate sentence with seacher
    output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
    # Format and print response sentence
    output_words[:] = [
        x for x in output_words if not (x == "EOS" or x == "PAD" or x=="SOS")
    ]
    print("\nBot(BeamSearch):", "".join(output_words))
    print("正解:",output_sentence+"\n\n")

入力: ▁ごめんねスムージー可愛すぎるわw初見にやけすぎてミスしすぎw
入力を分割すると: ▁ごめんね ス ムー ジー 可愛すぎる わ w 初見 にやけ すぎて ミス しすぎ w

Bot(BeamSearch): ▁wwwww
正解: ▁目の色違うから人違いかな!


入力: ▁あやて私だったり、?
入力を分割すると: ▁あや て 私 だったり 、 ?

Bot(BeamSearch): ▁え?w
正解: ▁暇すぎて暗闇の中ガンプラ作る始末


入力: ▁いいねで
入力を分割すると: ▁いいね で

Bot(BeamSearch): ▁
正解: ▁リドルジョーカーねぇなぁって


入力: ▁15時間寝てたwありがとう
入力を分割すると: ▁15 時間 寝てた w ありがとう

Bot(BeamSearch): ▁wwwwww
正解: ▁みんなに知られたまーくんであったー


入力: ▁楽しかったねーまた遊ぼあれよあれ。もうこたんめんのしんはつびー
入力を分割すると: ▁楽しかった ねー また 遊ぼ あれ よ あれ 。 もう こたん めん の しん は つ びー

Bot(BeamSearch): ▁wwwww
正解: ▁それは宜野座伸元でしょうかw




# 複数文を自動評価指標で評価、ここから下

In [46]:
eval_folder = 'examples'
from nlgeval import compute_metrics
metrics_dict = compute_metrics(hypothesis=eval_folder+'/hyp.txt',
                               references=[eval_folder+'/ref1.txt'])


Bleu_1: 0.550000
Bleu_2: 0.428174
Bleu_3: 0.284043
Bleu_4: 0.201143
METEOR: 0.295797
ROUGE_L: 0.522104
CIDEr: 2.001651
SkipThoughtsCosineSimilarity: 0.626149
EmbeddingAverageCosineSimilarity: 0.866522
EmbeddingAverageCosineSimilairty: 0.866522
VectorExtremaCosineSimilarity: 0.558104
GreedyMatchingScore: 0.779369


In [47]:
import pickle
# eval_folder = 'mecab3min'
eval_folder = 'sentencePiece28min'
# test_extract = [random.choice(test_pairs) for i in range(5000)]


# input_choices = [ele[0] for ele in test_extract]
# output_choices = [ele[1] for ele in test_extract]


# with open(eval_folder+'/eval_input_output.pickle', 'wb') as f:
#     pickle.dump((input_choices ,output_choices), f)

with open(eval_folder+"/eval_input_output.pickle", "rb") as f:
    
    input_choices ,output_choices= pickle.load(f)
    
print(input_choices[:10])

print(output_choices[:10])

['▁ 違った 意味で 有名 になりそう やわ w', '▁なんか 、 すべて において 、 聞いて らん なかったです w', '▁ 残り 2 日 応援 お願いします', '▁大丈夫 ! ロック は 宗教 だから そんな 気持ち パッパ ラ パー だよ !', '▁10 代 ボイス なのか ! なん という なん ということ !', '▁3 枚目 保存して 使って いい ?', '▁エロ オタク の友達 w', '▁元気 さんが 俺の 隣で にゃ にゃ に ー するんですか ? 笑 w', '▁大丈夫です !', '▁やっぱ そーなの !? w']
['▁ 江 田 島 みたいに やってほしい w 絶対 ピコ タン のほうが 面白い けど', '▁ やっちゃった なー が正解 かと w', '▁はい 頑張って', '▁ ロック は 宗教 そっか それなら 仕方ない 機会あれば 角 砂糖 焼肉 の タレ 添え を 試 そう', '▁声 、 格好いい よね', '▁ちょっとまって ! 比較 してくれ !', '▁あや のこと かな ?', '▁ お互いに w', '▁こちら 集計 となります ! 本日は対戦ありがとうございました !', '▁そう でしょ w あと 漢 方 なら 水 よりも 白 湯 が おすすめ b']


In [48]:
eval_folder

'sentencePiece28min'

In [49]:
# with open(eval_folder+'/ref.txt','w') as f:
#     f.write('\n'.join(output_choices))

### ファイル名注意

In [54]:
import pandas as pd

csv_file = 'to_csv_out4.csv'
df = pd.DataFrame(columns=list(metrics_dict.keys()))
df.to_csv(eval_folder+'/'+csv_file)
print(df)

Empty DataFrame
Columns: [Bleu_1, Bleu_2, Bleu_3, Bleu_4, METEOR, ROUGE_L, CIDEr, SkipThoughtCS, EmbeddingAverageCosineSimilarity, EmbeddingAverageCosineSimilairty, VectorExtremaCosineSimilarity, GreedyMatchingScore]
Index: []


In [55]:
import os
import re
# pathPos = './data/save/focal_loss_model/mecab_tagged.txt/4-4_256_0.2/'
pathPos = './data/save/focal_loss_model/train_model32000.txt/4-4_256_0.2/'
fileList = os.listdir(pathPos)
print(len(fileList))
files_file = [f for f in fileList if os.path.isfile(os.path.join(pathPos,f))]
print(len(files_file))
print(files_file)

checkpoint_list = []

for f in files_file:
    numStr = re.search(r'\d+',f)
    numStr = int(numStr.group())
    checkpoint_list.append(numStr)
checkpoint_list.sort()
print()
print(len(checkpoint_list))
print(checkpoint_list)

39
38
['9000_checkpoint.tar', '310000_checkpoint.tar', '410000_checkpoint.tar', '100000_checkpoint.tar', '20000_checkpoint.tar', '90000_checkpoint.tar', '250000_checkpoint.tar', '50000_checkpoint.tar', '285000_checkpoint.tar', '460000_checkpoint.tar', '360000_checkpoint.tar', '200000_checkpoint.tar', '8000_checkpoint.tar', '500000_checkpoint.tar', '330000_checkpoint.tar', '430000_checkpoint.tar', '380000_checkpoint.tar', '480000_checkpoint.tar', '70000_checkpoint.tar', '150000_checkpoint.tar', '440000_checkpoint.tar', '340000_checkpoint.tar', '490000_checkpoint.tar', '390000_checkpoint.tar', '350000_checkpoint.tar', '60000_checkpoint.tar', '450000_checkpoint.tar', '420000_checkpoint.tar', '10000_checkpoint.tar', '320000_checkpoint.tar', '80000_checkpoint.tar', '23000_checkpoint.tar', '370000_checkpoint.tar', '470000_checkpoint.tar', '7000_checkpoint.tar', '400000_checkpoint.tar', '30000_checkpoint.tar', '300000_checkpoint.tar']

38
[7000, 8000, 9000, 10000, 20000, 23000, 30000, 50000, 

In [56]:
from nlgeval import compute_metrics
import pandas as pd

print(corpus_name)
# Configure models
model_name = "focal_loss_model"
attn_model = 'dot'
# attn_model = 'general'
# attn_model = "concat"
hidden_size = 256
encoder_n_layers = 4
decoder_n_layers = 4
dropout = 0.1
batch_size = 512
teacher_forcing_ratio=0.2

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None

#1000,2000,3000,4000,5000,6000,7000,8000,9000,15000,30000,52000,56000,60000,64000,68000,72000,96000,100000,102000,104000,120000,124000,128000,129000,148000,150000,151000,152000,153000,154000,155000,156000,157000,158000
#159000,160000,161000,162000,163000,164000,165000,166000,167000,168000,169000,170000,171000,172000,173000
#174000,175000,176000,177000,178000,179000,180000,181000,182000,183000,184000,185000,186000,187000,188000,189000,190000,191000,192000,193000,194000,195000,196000,197000
# checkpoint_list = [4000,15000,30000,52000,56000,68000,72000,100000,104000,120000,124000,128000,148000,150000,151000,152000,153000,155000,156000,157000,158000,
#                    159000,160000,161000,162000,163000,164000,165000,166000,167000,168000,169000,170000,171000,172000,173000,174000,175000,176000,178000,179000,
#                    180000,181000,182000,183000,184000,185000,186000,187000,188000,189000,190000,191000,192000,193000,194000,195000,196000,197000]

for checkpoint_iter in checkpoint_list:
    
    print('test',checkpoint_iter,'iter')

    loadFilename = os.path.join(save_dir, model_name, corpus_name,
                               '{}-{}_{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size, teacher_forcing_ratio),
                               '{}_checkpoint.tar'.format(checkpoint_iter))


    # Load model if a loadFilename is provided
    if loadFilename:
        print("load save file")
        # If loading on same machine the model was trained on
        checkpoint = torch.load(loadFilename)
        # If loading a model trained on GPU to CPU
        # checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
        encoder_sd = checkpoint["en"]
        decoder_sd = checkpoint["de"]
        encoder_optimizer_sd = checkpoint["en_opt"]
        decoder_optimizer_sd = checkpoint["de_opt"]
        embedding_sd = checkpoint["embedding"]
        voc.__dict__ = checkpoint["voc_dict"]


    print("Building encoder and decoder ...")
    # Initialize word embeddings
    embedding = nn.Embedding(voc.num_words, hidden_size)
    if loadFilename:
        embedding.load_state_dict(embedding_sd)
    # Initialize encoder & decoder models
    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = AttnDecoderRNN(
        attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout
    )
    if loadFilename:
        encoder.load_state_dict(encoder_sd)
        decoder.load_state_dict(decoder_sd)
    # Use appropriate device
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    print("Models built and ready to go!")
    
    
    
    encoder.eval()
    decoder.eval()
    searcher = BeamSearchDecoder(encoder, decoder, 5, 5)
    hyp_sentences = []
    
    
    for input_sentence in input_choices:
        input_sentence = normalizeString(input_sentence)
        output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
        output_words[:] = [
            x for x in output_words if not (x == "EOS" or x == "PAD" or x=="SOS")
        ]
        hyp_sentences.append("".join(output_words))

        
    with open(eval_folder+'/hyp.txt','w') as f:
        f.write('\n'.join(hyp_sentences))
        
        
    
    metrics_dict = compute_metrics(hypothesis=eval_folder+'/hyp.txt',
                                   references=[eval_folder+'/ref.txt'])
    
    df = pd.read_csv(eval_folder+'/'+csv_file, index_col=0)
    nums = metrics_dict.values()
    df.loc[checkpoint_iter] = list(map(round,nums,[6]*len(nums)))
    df.to_csv(eval_folder+'/'+csv_file)


train_model32000.txt
test 7000 iter
load save file
Building encoder and decoder ...
Models built and ready to go!
Bleu_1: 0.000016
Bleu_2: 0.000000
Bleu_3: 0.000000
Bleu_4: 0.000000
METEOR: 0.032861
ROUGE_L: 0.002542
CIDEr: 0.001128
SkipThoughtsCosineSimilarity: 0.388682
EmbeddingAverageCosineSimilarity: 0.686857
EmbeddingAverageCosineSimilairty: 0.686857
VectorExtremaCosineSimilarity: 0.230561
GreedyMatchingScore: 0.884528
test 8000 iter
load save file
Building encoder and decoder ...
Models built and ready to go!
Bleu_1: 0.000105
Bleu_2: 0.000000
Bleu_3: 0.000000
Bleu_4: 0.000000
METEOR: 0.034706
ROUGE_L: 0.013108
CIDEr: 0.005267
SkipThoughtsCosineSimilarity: 0.381632
EmbeddingAverageCosineSimilarity: 0.635135
EmbeddingAverageCosineSimilairty: 0.635135
VectorExtremaCosineSimilarity: 0.240693
GreedyMatchingScore: 0.872019
test 9000 iter
load save file
Building encoder and decoder ...
Models built and ready to go!
Bleu_1: 0.000018
Bleu_2: 0.000000
Bleu_3: 0.000000
Bleu_4: 0.000000
METE

In [ ]:
Bleu_1: 0.003000
Bleu_2: 0.000055
Bleu_3: 0.000014
Bleu_4: 0.000007
METEOR: 0.036579
ROUGE_L: 0.003000
CIDEr: 0.007500
SkipThoughtsCosineSimilarity: 0.736541
EmbeddingAverageCosineSimilarity: 0.633790
EmbeddingAverageCosineSimilairty: 0.633790
VectorExtremaCosineSimilarity: 0.562120
GreedyMatchingScore: 0.868408